In [116]:
import pandas as pd
import os
import datetime
import json
import numpy as np

# File configuration
file_path = 'rss.xlsx'

# Time conversion helper functions
def time_string_to_minutes(time_str):
    """Convert time string (HH:MM:SS or MM:SS) to total minutes"""
    if pd.isna(time_str) or time_str == '' or time_str is None:
        return 0
    
    try:
        # Handle string format like "44:30:10" (HH:MM:SS)
        if isinstance(time_str, str):
            parts = time_str.split(':')
            if len(parts) == 3:  # HH:MM:SS
                hours, minutes, seconds = map(int, parts)
                return hours * 60 + minutes + seconds / 60
            elif len(parts) == 2:  # MM:SS
                minutes, seconds = map(int, parts)
                return minutes + seconds / 60
            else:
                return 0
        # Handle pandas Timedelta
        elif isinstance(time_str, pd.Timedelta):
            return time_str.total_seconds() / 60
        elif isinstance(time_str, datetime.time):
            return time_str.hour * 60 + time_str.minute + time_str.second / 60
        # Handle numpy timedelta64
        elif isinstance(time_str, np.timedelta64):
            return pd.Timedelta(time_str).total_seconds() / 60
        else:
            return 0
    except (ValueError, AttributeError):
        print(f"Warning: Could not parse time value: {time_str}")
        return 0

def timedelta_to_minutes(td):
    """Convert pandas Timedelta or datetime.time to minutes"""
    if pd.isna(td):
        return 0
    
    # Handle pandas Timedelta
    if isinstance(td, pd.Timedelta):
        return td.total_seconds() / 60
    
    # Handle numpy timedelta64
    elif isinstance(td, np.timedelta64):
        return pd.Timedelta(td).total_seconds() / 60
    
    # Handle datetime.time objects (treat as duration from 00:00:00)
    elif isinstance(td, datetime.time):
        total_seconds = td.hour * 3600 + td.minute * 60 + td.second + td.microsecond / 1000000
        return total_seconds / 60
    
    # Handle string format
    elif isinstance(td, str):
        return time_string_to_minutes(td)
    
    else:
        print(f"Warning: Unknown time type {type(td)} for value {td}")
        return 0

# JSON serialization helper for pandas objects
def make_json_serializable(obj):
    """Convert pandas/numpy objects to JSON serializable formats"""
    if isinstance(obj, dict):
        return {key: make_json_serializable(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [make_json_serializable(item) for item in obj]
    elif isinstance(obj, pd.Timedelta):
        return obj.total_seconds() / 60  # Convert timedelta to minutes
    elif isinstance(obj, datetime.timedelta):  # Handle Python native timedelta
        return obj.total_seconds() / 60  # Convert timedelta to minutes
    elif isinstance(obj, np.timedelta64):
        return pd.Timedelta(obj).total_seconds() / 60  # Convert numpy timedelta to minutes
    elif isinstance(obj, datetime.time):  # Handle datetime.time objects
        return timedelta_to_minutes(obj)  # Convert time to minutes
    elif isinstance(obj, pd.Timestamp):
        return obj.isoformat()  # Convert timestamp to ISO format string
    elif isinstance(obj, np.datetime64):
        return pd.Timestamp(obj).isoformat()
    elif isinstance(obj, (np.int64, np.int32, np.int16, np.int8)):
        return int(obj)  # Convert numpy integers to Python int
    elif isinstance(obj, (np.float64, np.float32)):
        return float(obj)  # Convert numpy floats to Python float
    elif pd.isna(obj):
        return None  # Convert NaN/NaT to None
    else:
        return obj
    
# Safe JSON export - handles pandas/numpy objects
def safe_json_export(data, filename):
    """Safely export data to JSON, handling pandas/numpy objects"""
    # Create output directory if it doesn't exist
    os.makedirs('output', exist_ok=True)
    
    # Convert data to JSON serializable format
    serializable_data = make_json_serializable(data)
    
    # Write to file
    output_path = f'output/{filename}'
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(serializable_data, f, indent=2, ensure_ascii=False)
    
    print(f"Successfully exported to {output_path}")
    return output_path



In [117]:
# Check if file exists and read all sheets at once (more efficient)
if not os.path.exists(file_path):
    print(f"File {file_path} not found!")
    all_sheets = None
else:
    print(f"Reading all sheets from {file_path}...")
    # Read all sheets at once - this is more efficient than reading each sheet separately
    all_sheets = pd.read_excel(file_path, sheet_name=None)
    
    print(f"Successfully loaded {len(all_sheets)} sheets:")
    for sheet_name in all_sheets.keys():
        print(f"  - {sheet_name}: {all_sheets[sheet_name].shape}")


Reading all sheets from rss.xlsx...
Successfully loaded 60 sheets:
  - queen: (26, 18)
  - woodlouse_colony: (27, 11)
  - meat_depot: (12, 11)
  - meat_depot_2: (12, 11)
  - meat_depot_3: (12, 11)
  - spring: (27, 11)
  - reservoir: (12, 11)
  - reservoir_2: (12, 11)
  - reservoir_3: (12, 11)
  - plant_flora: (27, 11)
  - plant_depot: (12, 11)
  - plant_depot_2: (12, 11)
  - plant_depot_3: (12, 11)
  - wet_soil_pile: (27, 11)
  - wet_soil_depot: (12, 11)
  - wet_soil_depot_2: (12, 11)
  - wet_soil_depot_3: (12, 11)
  - sand_pile: (27, 11)
  - sand_depot: (12, 11)
  - sand_depot_2: (12, 11)
  - sand_depot_3: (12, 11)
  - leafcutter: (27, 11)
  - fungus_depot: (12, 11)
  - fungus_depot_2: (12, 11)
  - fungus_depot_3: (12, 11)
  - native_fungi: (27, 11)
  - supreme_native_fungi: (27, 11)
  - worker_ant_nest: (27, 11)
  - feeding_ground: (27, 12)
  - aphid: (27, 13)
  - ladybug_habitat: (27, 11)
  - resource_factory: (27, 11)
  - resource_tunnel: (27, 11)
  - trophy_storeroom: (27, 11)
  -

In [118]:
# Filter sheets if needed (example: only sheets containing specific text)
if all_sheets is not None:
    # Example: filter sheets by name pattern
    # filtered_sheets = {name: df for name, df in all_sheets.items() if 'Особые постройки' in name}
    
    # Or use all sheets
    filtered_sheets = all_sheets
    
    print(f"\nWorking with {len(filtered_sheets)} sheet(s):")
    for sheet_name in filtered_sheets.keys():
        print(f"  - {sheet_name}")
else:
    filtered_sheets = {}



Working with 60 sheet(s):
  - queen
  - woodlouse_colony
  - meat_depot
  - meat_depot_2
  - meat_depot_3
  - spring
  - reservoir
  - reservoir_2
  - reservoir_3
  - plant_flora
  - plant_depot
  - plant_depot_2
  - plant_depot_3
  - wet_soil_pile
  - wet_soil_depot
  - wet_soil_depot_2
  - wet_soil_depot_3
  - sand_pile
  - sand_depot
  - sand_depot_2
  - sand_depot_3
  - leafcutter
  - fungus_depot
  - fungus_depot_2
  - fungus_depot_3
  - native_fungi
  - supreme_native_fungi
  - worker_ant_nest
  - feeding_ground
  - aphid
  - ladybug_habitat
  - resource_factory
  - resource_tunnel
  - trophy_storeroom
  - pro_rally_center
  - evolution_fungi
  - special_nest
  - special_ant_habitat
  - mutation_pool
  - construction_center
  - guardian_ant_nest
  - shooter_ant_nest
  - carrier_ant_nest
  - healing_pool
  - toxic_fungi
  - sentinel_tree
  - cocoon_medium
  - rally_center_i
  - rally_center_ii
  - rally_center_iii
  - entrance
  - mutation_flora
  - soldiers_reform_pool
  - treas

In [119]:
# Explore sheet data - basic information
for sheet_name, df in filtered_sheets.items():
    print(f"\n{'='*60}")
    print(f"SHEET: {sheet_name}")
    print(f"{'='*60}")
    print(f"Shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    print(f"\nFirst 3 rows:")
    print(df.head(3))
    print(f"\nData types:")
    print(df.dtypes)
    print(f"\nNull values per column:")
    print(df.isnull().sum())



SHEET: queen
Shape: (26, 18)
Columns: ['level', 'meat', 'plant', 'fungus', 'wet_soil', 'sand', 'honeydew', 'time', 'Диас', 'Хоры', 'power_delta__', 'time_d', 'diamonds', 'power', 'power_delta', 'population', 'requirement_1', 'requirement_2']

First 3 rows:
  level  meat  plant  fungus  wet_soil  sand  honeydew            time  \
0     1     0      0       0         0     0         0             NaT   
1     2     0    200       0         0     0         0 0 days 00:00:03   
2     3   100    200       0         0     0         0 0 days 00:00:50   

       Диас  Хоры  power_delta__ time_d  diamonds   power  power_delta  \
0         -   NaN            NaN      0         0     0.0          0.0   
1  0.000035   0.0       0.000035   0d0h         1  2568.0       2568.0   
2  0.000579   0.0       0.000579   0d0h         4  2773.0        205.0   

   population       requirement_1        requirement_2  
0         0.0                 NaN                  NaN  
1         0.0                 NaN 

In [120]:
# Processing cell - Add your custom logic here
processed_data = {}


for sheet_name, df in filtered_sheets.items():
    print(f"\nProcessing sheet: {sheet_name}")
    

    build_info = {
        "$schema": "../building-schema.json",
        'id': sheet_name,
        'levels': [],
            'warns': []
    }

    # Iterate over rows in the dataframe
    for index, row in df.iterrows():
        level_info = {
            'level': index + 1,  # Assuming levels start from 1
            'meat': 0,
            'fungus': 0,
            'plant': 0,
            'wet_soil': 0,
            'sand': 0,
            'diamonds': 0,
            'power': 0,
            'power_delta': 0,
            'population': 0,
            'honeydew': 0,
            'time': 0,
            'notes': '',
            'requirements': {
                'queen': index + 1
            }
        }
        
        # Process each column in the row
        for column in df.columns:
            value = row[column]
            if column not in level_info:
                build_info['warns'].append(f"Column {column} not found in level_info")
                continue
            if pd.notna(value):  # Only include non-null values
                # Special handling for time column
                if column == 'time':
                    level_info[column] = timedelta_to_minutes(value)
                else:
                    level_info[column] = value
        
        build_info['levels'].append(level_info)


    # Use safe JSON export to handle pandas/numpy objects
    filename = f"{sheet_name.replace('/', '_')}.json"  # Replace invalid filename chars
    safe_json_export(build_info, filename)
    
    # Store processed data (using the original dataframe since we're extracting build_info)
    processed_data[sheet_name] = df
    
    print(f"  - Original shape: {df.shape}")
    print(f"  - Processed {len(build_info['levels'])} levels")

print(f"\nProcessing complete! Processed {len(processed_data)} sheets.")



Processing sheet: queen
Successfully exported to output/queen.json
  - Original shape: (26, 18)
  - Processed 26 levels

Processing sheet: woodlouse_colony
Successfully exported to output/woodlouse_colony.json
  - Original shape: (27, 11)
  - Processed 27 levels

Processing sheet: meat_depot
Successfully exported to output/meat_depot.json
  - Original shape: (12, 11)
  - Processed 12 levels

Processing sheet: meat_depot_2
Successfully exported to output/meat_depot_2.json
  - Original shape: (12, 11)
  - Processed 12 levels

Processing sheet: meat_depot_3
Successfully exported to output/meat_depot_3.json
  - Original shape: (12, 11)
  - Processed 12 levels

Processing sheet: spring
Successfully exported to output/spring.json
  - Original shape: (27, 11)
  - Processed 27 levels

Processing sheet: reservoir
Successfully exported to output/reservoir.json
  - Original shape: (12, 11)
  - Processed 12 levels

Processing sheet: reservoir_2
Successfully exported to output/reservoir_2.json
  -

In [121]:
# Access individual sheets for specific processing
# Example: work with a specific sheet
if processed_data:
    # Get sheet names
    sheet_names = list(processed_data.keys())
    print(f"Available sheets: {sheet_names}")
    
    # Access a specific sheet by name
    if sheet_names:
        first_sheet_name = sheet_names[0]
        first_sheet = processed_data[first_sheet_name]
        
        print(f"\nWorking with sheet: {first_sheet_name}")
        print(f"Shape: {first_sheet.shape}")
        
        # Example: specific processing for this sheet
        # your_specific_logic_here = first_sheet.groupby('column').sum()
        
        # You can now work with individual sheets as needed
        # sheet_2 = processed_data['Sheet2'] if 'Sheet2' in processed_data else None


Available sheets: ['queen', 'woodlouse_colony', 'meat_depot', 'meat_depot_2', 'meat_depot_3', 'spring', 'reservoir', 'reservoir_2', 'reservoir_3', 'plant_flora', 'plant_depot', 'plant_depot_2', 'plant_depot_3', 'wet_soil_pile', 'wet_soil_depot', 'wet_soil_depot_2', 'wet_soil_depot_3', 'sand_pile', 'sand_depot', 'sand_depot_2', 'sand_depot_3', 'leafcutter', 'fungus_depot', 'fungus_depot_2', 'fungus_depot_3', 'native_fungi', 'supreme_native_fungi', 'worker_ant_nest', 'feeding_ground', 'aphid', 'ladybug_habitat', 'resource_factory', 'resource_tunnel', 'trophy_storeroom', 'pro_rally_center', 'evolution_fungi', 'special_nest', 'special_ant_habitat', 'mutation_pool', 'construction_center', 'guardian_ant_nest', 'shooter_ant_nest', 'carrier_ant_nest', 'healing_pool', 'toxic_fungi', 'sentinel_tree', 'cocoon_medium', 'rally_center_i', 'rally_center_ii', 'rally_center_iii', 'entrance', 'mutation_flora', 'soldiers_reform_pool', 'treasure_depot', 'alliance_center', 'troop_tunnel', 'insect_habitat',

In [122]:
# Test time conversion - Debug your time parsing issue
test_time_string = "44:30:10"
print(f"Testing time string: {test_time_string}")

# Method 1: Using our helper function
minutes_1 = time_string_to_minutes(test_time_string)
print(f"Method 1 (helper function): {minutes_1} minutes")

# Method 2: Manual calculation
parts = test_time_string.split(':')
hours, minutes, seconds = map(int, parts)
total_minutes = hours * 60 + minutes + seconds / 60
print(f"Method 2 (manual): {total_minutes} minutes")

# Method 3: Using pandas to_timedelta (if your data is in timedelta format)
try:
    # Convert string to timedelta first
    td = pd.to_timedelta(test_time_string)
    minutes_3 = td.total_seconds() / 60
    print(f"Method 3 (pandas timedelta): {minutes_3} minutes")
    print(f"Timedelta object: {td}")
except Exception as e:
    print(f"Method 3 failed: {e}")

# Check what type your actual time column is
if 'queen' in filtered_sheets:
    sample_time = filtered_sheets['queen']['time'].iloc[2]  # Get a non-null time value
    print(f"\nActual time value from data: {sample_time}")
    print(f"Type: {type(sample_time)}")
    print(f"Converted to minutes: {timedelta_to_minutes(sample_time)}")
    
    # Test with datetime.time object directly
    import datetime
    test_time_obj = datetime.time(hour=2, minute=30, second=10)  # 2:30:10
    print(f"\nTest datetime.time object: {test_time_obj}")
    print(f"Converted to minutes: {timedelta_to_minutes(test_time_obj)}")  # Should be 150.167 minutes
    
    # If your data shows something like 44:30:10, it might be stored incorrectly
    # Let's check a few more samples from the time column
    print(f"\nFirst 5 time values from queen sheet:")
    for i, time_val in enumerate(filtered_sheets['queen']['time'].head()):
        print(f"  {i}: {time_val} (type: {type(time_val)}) -> {timedelta_to_minutes(time_val)} minutes")


Testing time string: 44:30:10
Method 1 (helper function): 2670.1666666666665 minutes
Method 2 (manual): 2670.1666666666665 minutes
Method 3 (pandas timedelta): 2670.1666666666665 minutes
Timedelta object: 1 days 20:30:10

Actual time value from data: 0 days 00:00:50
Type: <class 'pandas._libs.tslibs.timedeltas.Timedelta'>
Converted to minutes: 0.8333333333333334

Test datetime.time object: 02:30:10
Converted to minutes: 150.16666666666666

First 5 time values from queen sheet:
  0: NaT (type: <class 'pandas._libs.tslibs.nattype.NaTType'>) -> 0 minutes
  1: 0 days 00:00:03 (type: <class 'pandas._libs.tslibs.timedeltas.Timedelta'>) -> 0.05 minutes
  2: 0 days 00:00:50 (type: <class 'pandas._libs.tslibs.timedeltas.Timedelta'>) -> 0.8333333333333334 minutes
  3: 0 days 00:02:00 (type: <class 'pandas._libs.tslibs.timedeltas.Timedelta'>) -> 2.0 minutes
  4: 0 days 00:11:50 (type: <class 'pandas._libs.tslibs.timedeltas.Timedelta'>) -> 11.833333333333334 minutes


In [123]:
# Debug: Check for problematic data types in your build_info
def find_non_serializable_objects(obj, path=""):
    """Recursively find objects that might not be JSON serializable"""
    problematic_types = []
    
    if isinstance(obj, dict):
        for key, value in obj.items():
            problematic_types.extend(find_non_serializable_objects(value, f"{path}.{key}"))
    elif isinstance(obj, list):
        for i, item in enumerate(obj):
            problematic_types.extend(find_non_serializable_objects(item, f"{path}[{i}]"))
    else:
        # Check for types that might cause issues
        if isinstance(obj, (pd.Timedelta, datetime.timedelta, np.timedelta64, datetime.time)):
            problematic_types.append((path, type(obj).__name__, str(obj)))
        elif hasattr(obj, 'dtype') and 'datetime' in str(obj.dtype):
            problematic_types.append((path, type(obj).__name__, str(obj)))
    
    return problematic_types

# Test the debug function with a sample from your data
if 'queen' in filtered_sheets:
    # Get a sample row to build test data
    sample_row = filtered_sheets['queen'].iloc[2]
    
    sample_build_info = {
        'id': 'test',
        'levels': [{
            'level': 1,
            'time': sample_row['time'],
            'meat': sample_row['meat'],
            'other_data': 'test'
        }],
        'warns': []
    }
    
    print("Checking for problematic objects in sample data:")
    problems = find_non_serializable_objects(sample_build_info)
    for path, obj_type, value in problems:
        print(f"  {path}: {obj_type} = {value}")
    
    if not problems:
        print("  No problematic objects found!")
    
    # Test the serialization
    print(f"\nTesting serialization:")
    try:
        serialized = make_json_serializable(sample_build_info)
        print("  ✅ Serialization successful!")
        print(f"  Time value converted to: {serialized['levels'][0]['time']} minutes")
    except Exception as e:
        print(f"  ❌ Serialization failed: {e}")


Checking for problematic objects in sample data:
  .levels[0].time: Timedelta = 0 days 00:00:50

Testing serialization:
  ✅ Serialization successful!
  Time value converted to: 0.8333333333333334 minutes
